<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/query_transformations/03_react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent - A Simple Intro with Calculator Tools

This is a notebook that showcases the ReAct agent over very simple calculator tools (no fancy RAG pipelines or API calls).

We show how it can reason step-by-step over different tools to achieve the end goal.

In [ ]:
!pip install boto3
!pip install llama-index
!pip install llama-index-llms-bedrock

In [2]:
import os
import boto3

# Set AWS env config
region_name = os.getenv("AWS_REGION", "us-east-1")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID", "")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY", "")

In [3]:
from llama_index.llms.bedrock import Bedrock
from llama_index.core import Settings

llm_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

llm = Bedrock(
    model=llm_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    temperature=0.1,
    max_tokens=512
    )

Settings.llm = llm

## Define Function Tools

We setup some trivial `multiply` and `add` tools. Note that you can define arbitrary functions and pass it to the `FunctionTool` (which will process the docstring and parameter signature).

In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [5]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [6]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Run Some Queries

In [7]:
agent = ReActAgent.from_tools(
    [
        multiply_tool,
        add_tool
    ],
    llm=llm,
    verbose=True
    )

In [8]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

Thought: The current language of the user is: English. I need to use tools to help me answer the question step by step.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I need to add 20 and the result from the previous multiplication to get the final answer.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: To calculate 20+(2*4) step by step:

1. First, we calculate 2*4 using the multiply tool:
   2 * 4 = 8

2. Then, we add 20 and the result from step 1 using the add tool: 
   20 + 8 = 28

Therefore, the answer to 20+(2*4) is 28.


## View Prompts

Let's take a look at the core system prompt powering the ReAct agent!

Within the agent, the current conversation history is dumped below this line.

In [10]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t

### Customizing the Prompt

For fun, let's try instructing the agent to output the answer along with reasoning in bullet points. See "## Additional Rules" section.

In [11]:
from llama_index.core import PromptTemplate

react_system_header_str = """\

You are designed to help with a variety of tasks, from answering questions \
    to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
react_system_prompt = PromptTemplate(react_system_header_str)

In [12]:
agent.get_prompts()

{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'tool_names'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the question.\nAction: tool name (one of {tool_names}) if using a tool.\nAction Input: the i

In [13]:
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [14]:
agent.reset()
response = agent.chat("What is 5+3+2")
print(response)

Thought: To calculate 5+3+2, I can break it down into two steps: first add 5 and 3, then add the result to 2.
Action: add
Action Input: {'a': 5, 'b': 3}
Observation: 8
Thought: Now I need to add the result of 5+3, which is 8, to 2.
Action: add
Action Input: {'a': 8, 'b': 2}
Observation: 10
Thought: I now have the result of 5+3+2, which is 10.
Answer: The answer to 5+3+2 is 10. Here are the steps:

- First, I added 5 and 3 using the add tool, which gave 8.
- Then, I added 8 and 2 using the add tool again, which gave the final result of 10.
The answer to 5+3+2 is 10. Here are the steps:

- First, I added 5 and 3 using the add tool, which gave 8.
- Then, I added 8 and 2 using the add tool again, which gave the final result of 10.
